In [4]:
#This loads required libraries. 
import importlib
import ruamel.yaml
import pymongo
from pymongo import MongoClient#A .yaml file
cf_file="vm.yaml"
with open(cf_file, 'r') as yaml:
    cf=ruamel.yaml.round_trip_load(yaml, preserve_quotes=True)
client = MongoClient(cf['mongo_uri'])
db = client[cf['mongo_db']]

#process = cp.CrunchBaseProcessor('mac.yaml')
#total=process.count_mongo()

In [5]:
import pandas as pd
#This obtains all documents in a collection.  In the "find" is where you have a specific query. 
collection = db.acquisitions
#Tutorial
#https://docs.mongodb.com/manual/tutorial/project-fields-from-query-results/
cursor = collection.find({ }, { 'data.relationships.acquiree.item.uuid': 1, 'data.relationships.acquiree.item.properties.permalink': 1,  
                                'data.relationships.acquirer.item.uuid': 1, 'data.relationships.acquirer.item.properties.permalink': 1, 
                                '_id':0} )

In [6]:
cursor_df = pd.DataFrame(list(cursor))
cursor_df.head()

,data
0,{'relationships': {'acquiree': {'item': {'uuid...
1,{'relationships': {'acquiree': {'item': {'uuid...
2,{'relationships': {'acquiree': {'item': {'uuid...
3,{'relationships': {'acquiree': {'item': {'uuid...
4,{'relationships': {'acquiree': {'item': {'uuid...


In [10]:
# df_all = pd.DataFrame(columns=['acquiree_uuid','acquiree_permalink','acquirer_uuid','acquirer_permalink'])
# for document in cursor_df['data']:
#     df_all = df_all.append({'acquiree_uuid':document['relationships']['acquiree']['item']['uuid'],
#                    'acquiree_permalink':document['relationships']['acquiree']['item']['properties']['permalink'],
#                    'acquirer_uuid':document['relationships']['acquirer']['item']['uuid'],
#                    'acquirer_permalink':document['relationships']['acquirer']['item']['properties']['permalink']},
#                     ignore_index=True)

# df_all.head()

In [13]:
cursor_df['acquiree_uuid'] = cursor_df['data'].apply(lambda x: x['relationships']['acquiree']['item']['uuid'])
cursor_df['acquiree_permalink'] = cursor_df['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['permalink'])
cursor_df['acquirer_uuid'] = cursor_df['data'].apply(lambda x: x['relationships']['acquirer']['item']['uuid'])
cursor_df['acquirer_permalink'] = cursor_df['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['permalink'])
cursor_df.head()

,data,acquiree_uuid,acquiree_permalink,acquirer_uuid,acquirer_permalink
0,{'relationships': {'acquiree': {'item': {'uuid...,b241aba65c5634767ef0026ea8921ddb,onyx-pharmaceuticals,42cd77d59fd6a2ce63ec45eebd518709,amgen-2
1,{'relationships': {'acquiree': {'item': {'uuid...,f0a575ab48245c74b8eb5e79a2e851da,’compass-solutions,d1c4cc8d833144830a1a583abb11c794,alphapoint-technology
2,{'relationships': {'acquiree': {'item': {'uuid...,fcff20f257e979915b208238770328f2,comfort-living,063ec1d62bc5dbf0bc6d0686ff6548a4,more-com
3,{'relationships': {'acquiree': {'item': {'uuid...,4746a915f5f0445aa3a6b90db3e71c5f,afw-uk-oil-gas,5a3eec7f01b834d2e235b95751cf7983,worleyparsons
4,{'relationships': {'acquiree': {'item': {'uuid...,b19d036149d9d8105471eef3fb36ce34,jt-usa-inc,c9b88a3c4e97f4b0dc128ce22b62775e,brass-eagle


In [16]:
cursor_df.drop(['data'],axis=1).to_csv('task1_uuid&permalink.csv')